# Automated analysis of geochemical data. 

## Statistical Analysis

### Block 1 : Imports and Environment Setup

In [ ]:
# --- Imports and basic configuration ---
from pathlib import Path
import os, math, warnings, json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
from scipy import stats
from scipy.stats import skew, kurtosis

# For better display in Jupyter
from IPython.display import display

# Reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Plot style
plt.rcParams['figure.figsize'] = (12, 6)

# --- Output directories ---
DATA_DIR = Path("C:/Folder") # add your folder here
INPUT_CSV = DATA_DIR / "Data.csv" # add your data source here

OUT_DIR_PNG = Path("./PNG"); OUT_DIR_PNG.mkdir(parents=True, exist_ok=True)
OUT_DIR_SHP = Path("./SHP"); OUT_DIR_SHP.mkdir(parents=True, exist_ok=True)
OUT_DIR_CSV = Path("./CSV"); OUT_DIR_CSV.mkdir(parents=True, exist_ok=True)

print("✅ Imports and directories are ready.")

### Block 2: Data Loading and Initial Preprocessing

In [ ]:
# --- Load CSV and rename coordinate columns ---
df = pd.read_csv(INPUT_CSV)

# Rename coordinate columns for consistency
rename_map = {"EAST":"x", "NORTH":"y", "RL":"z"} # rename coordinate columns if applicable
df = df.rename(columns=rename_map)

# Identify geochemical element columns by excluding metadata
exclude_cols = ['PROJECT', 'SAMPLEID', 'x', 'y', 'z'] # exclude useless columns if applicable
elements = [c for c in df.columns if c not in exclude_cols]
unique_elements_count = len(set(elements))

# Basic info
print(f"✅ Rows loaded: {len(df):,}")
print(f"ℹ️ Unique SAMPLEIDs: {df['SAMPLEID'].nunique():,}" if 'SAMPLEID' in df.columns else f"ℹ️ Total samples: {len(df):,}")
print("ℹ️ Unique geochemical elements:", unique_elements_count)

### Block 3: Missing Values and Outlier Detection

In [ ]:
# --- 3.1 Missing values in element columns ---
missing = df[elements].isna().sum()
missing = missing[missing > 0].sort_values(ascending=False)

if len(missing):
    print("🔍 Missing values in element columns (top 15):")
    display(missing.head(15))
else:
    print("✅ No missing values detected in element columns.")

# --- 3.2 Outlier flags: z-score and IQR ---
z_thresh = 3
z_input = df[elements].copy().fillna(df[elements].median(numeric_only=True))
z_mask = np.abs(stats.zscore(z_input, nan_policy='omit')) > z_thresh
df['flag_z_outlier'] = pd.DataFrame(z_mask, index=z_input.index, columns=z_input.columns).any(axis=1)

Q1 = df[elements].quantile(0.25)
Q3 = df[elements].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
iqr_mask = ((df[elements] < lower) | (df[elements] > upper)).fillna(False)
df['flag_iqr_outlier'] = iqr_mask.any(axis=1)

print(f"🚩 Rows with z-score outliers: {int(df['flag_z_outlier'].sum())}")
print(f"🚩 Rows with IQR outliers: {int(df['flag_iqr_outlier'].sum())}")


### Block 4: Standardizing Units to ppm (if applicable)

In [ ]:
# --- Helper: extract unit suffix from column name ---
def parse_unit(col: str) -> str:
    """
    Extracts unit suffix from column name: 'ppm', 'ppb', 'pct', or ''.
    Examples:
        'Cu_ppm' → 'ppm'
        'Au_ppb' → 'ppb'
        'Al_pct' → 'pct'
    """
    return col.rsplit('_', 1)[-1] if '_' in col else ''

# --- Convert selected columns to ppm ---
def to_ppm_frame(df_in: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    """
    Converts specified columns to ppm:
    - ppb → /1000
    - pct → *1e4
    - ppm or unknown → unchanged
    Prints list of converted columns.
    """
    out = pd.DataFrame(index=df_in.index)
    converted = []
    for c in cols:
        s = pd.to_numeric(df_in[c], errors='coerce')
        u = parse_unit(c)
        if u == 'ppb':
            out[c] = s / 1000.0
            converted.append(f"{c} (ppb → ppm)")
        elif u == 'pct':
            out[c] = s * 1e4
            converted.append(f"{c} (% → ppm)")
        else:
            out[c] = s
    if converted:
        print("✅ Converted to ppm:", ", ".join(converted))
    else:
        print("ℹ️ All elements already in ppm or unitless (no conversion needed).")
    return out

# --- Rename columns to reflect ppm conversion ---
def rename_to_ppm_columns(df_ppm: pd.DataFrame):
    """
    Renames columns with original units (ppb, pct) to *_ppm.
    Example:
        'Au_ppb' → 'Au_ppm'
        'Al_pct' → 'Al_ppm'
    If target name already exists, adds suffix '_from_<unit>'.
    Returns (renamed_df, name_map).
    """
    name_map = {}
    for c in df_ppm.columns:
        unit = parse_unit(c)
        if unit in ('ppb', 'pct'):
            base = c.rsplit('_', 1)[0]
            new_name = f"{base}_ppm"
            if new_name in df_ppm.columns and new_name != c:
                new_name = f"{base}_ppm_from_{unit}"
            name_map[c] = new_name
    df_renamed = df_ppm.rename(columns=name_map)
    return df_renamed, name_map

# --- Apply conversion and renaming ---
df_elems_ppm = to_ppm_frame(df, elements)
df_elems_ppm_named, ppm_name_map = rename_to_ppm_columns(df_elems_ppm)

if ppm_name_map:
    print("🔥 Renamed columns for clarity (analytics):")
    for old, new in ppm_name_map.items():
        print(f" {old} → {new}")
else:
    print("ℹ️ No renaming needed: all element columns already in *_ppm format.")

### Block 5: CLR Transformation and Basic Statistics

In [ ]:
# --- CLR transformation ---
def clr_transform(df_sub: pd.DataFrame, pseudocount: float = 1e-6) -> pd.DataFrame:
    """
    Performs classical CLR (Centered Log-Ratio) transformation:
    ln(x + pseudocount) - mean(ln(x + pseudocount)) across each row.
    Returns a DataFrame with columns named '<element>_CLR'.
    """
    arr = df_sub.fillna(0).values.astype(float) + pseudocount
    log_arr = np.log(arr)
    gm = log_arr.mean(axis=1)
    clr_arr = log_arr - gm[:, None]
    return pd.DataFrame(clr_arr, index=df_sub.index,
                        columns=[f"{c}_CLR" for c in df_sub.columns])

# --- Apply CLR transformation to renamed ppm element set ---
df_clr_ppm_named = clr_transform(df_elems_ppm_named)

# --- Calculate mean (mu) and standard deviation (sigma) for CLR-transformed columns ---
stats_df = pd.DataFrame(index=df_clr_ppm_named.columns)
stats_df['mu'] = df_clr_ppm_named.mean()
stats_df['sigma'] = df_clr_ppm_named.std(ddof=1)

print("✅ CLR transformation completed.")
# display(stats_df.head(10))

### Block 6: Variability Metrics — Robust CV, SD(log), Aitchison Variance

In [ ]:
# --- Robust CV (RAW, %) using MAD/median ---
def robust_cv_raw(series: pd.Series, scale_mad: bool = True) -> float:
    """
    Calculates robust coefficient of variation (CV) in percent:
    CV (%) = 100 * (k * MAD / median)
    Suitable for raw ppm values.
    """
    x = pd.to_numeric(series, errors='coerce').dropna().to_numpy()
    if x.size == 0:
        return np.nan
    med = float(np.median(x))
    if med == 0:
        return np.nan
    mad = float(np.median(np.abs(x - med)))
    k = 1.4826 if scale_mad else 1.0
    return 100.0 * (k * mad / med)

# --- SD(log x) for positive ppm values ---
def sd_log(series: pd.Series) -> float:
    """
    Computes standard deviation of log-transformed values.
    """
    x = pd.to_numeric(series, errors='coerce')
    x = x[x > 0]
    if x.size < 2:
        return np.nan
    y = np.log(x.to_numpy(dtype=float))
    return float(np.std(y, ddof=1))

# --- Aitchison variance for CLR-transformed components ---
def aitchison_var_component(clr_series: pd.Series) -> float:
    """
    Computes Aitchison variance of CLR-transformed component.
    """
    z = pd.to_numeric(clr_series, errors='coerce').dropna().to_numpy()
    if z.size < 2:
        return np.nan
    return float(np.var(z, ddof=1))

# --- Compute metrics ---
cv_raw_df = pd.DataFrame({
    'CV_%': {col: robust_cv_raw(df_elems_ppm_named[col], scale_mad=True)
             for col in df_elems_ppm_named.columns}
})

sd_log_df = pd.DataFrame({
    'SD_log': {col: sd_log(df_elems_ppm_named[col])
               for col in df_elems_ppm_named.columns}
})

aitch_var_df = pd.DataFrame({
    'AitchisonVar': {
        col.replace('_CLR', ''): aitchison_var_component(df_clr_ppm_named[col])
        for col in df_clr_ppm_named.columns
    }
})

print("🏁 Top 20 by Robust CV (RAW, %):")
display(cv_raw_df.sort_values('CV_%', ascending=False).head(20))

print("🏁 Top 20 by SD(log x):")
display(sd_log_df.sort_values('SD_log', ascending=False).head(20))

print("🏁 Top 20 by Aitchison variance (CLR):")
display(aitch_var_df.sort_values('AitchisonVar', ascending=False).head(20))

### Block 7: Exporting Metrics and Element Ranking

In [ ]:
# --- Helper: export metric to CSV ---
def _export_metric(df_onecol: pd.DataFrame, col_name: str, fname: str) -> pd.DataFrame:
    """
    Exports a single-column metric DataFrame to CSV with two columns: Element, <metric>.
    Returns the sorted DataFrame (descending).
    """
    out = (
        df_onecol.rename_axis('Element')
        .reset_index()
        .sort_values(col_name, ascending=False)
    )
    out.to_csv(OUT_DIR_CSV / fname, index=False, encoding="utf-8-sig")
    return out

# --- Helper: get top-k elements by metric ---
def _topk(df_onecol: pd.DataFrame, col_name: str, k: int = 15) -> pd.DataFrame:
    """
    Returns top-k rows from DataFrame by column col_name (descending).
    """
    return df_onecol.nlargest(min(k, len(df_onecol)), columns=col_name)

# --- Export each metric to CSV ---
cv_raw_out = _export_metric(cv_raw_df, 'CV_%', "cv_raw_ppm_robust_mad_median.csv")
sd_log_out = _export_metric(sd_log_df, 'SD_log', "sd_log_ppm.csv")
aitch_out = _export_metric(aitch_var_df, 'AitchisonVar', "aitchison_variance_clr.csv")

print("📁 Metrics exported to CSV:")
print(f" • Robust CV (RAW, %) → {OUT_DIR_CSV / 'cv_raw_ppm_robust_mad_median.csv'}")
print(f" • SD(log x) (LOG) → {OUT_DIR_CSV / 'sd_log_ppm.csv'}")
print(f" • AitchisonVar (CLR) → {OUT_DIR_CSV / 'aitchison_variance_clr.csv'}")

### Block 8: Integrated Element Ranking by Mean Metric Rank

In [ ]:
# --- Combine metrics into a single DataFrame ---
cv_comparison = (
    cv_raw_out[['Element', 'CV_%']]
    .merge(sd_log_out[['Element', 'SD_log']], on='Element', how='outer')
    .merge(aitch_out[['Element', 'AitchisonVar']], on='Element', how='outer')
)

# --- Rank elements by each metric (higher value = better rank) ---
rank_df = cv_comparison.copy()
rank_df['Rank_CV'] = rank_df['CV_%'].rank(ascending=False, method='min')
rank_df['Rank_SDlog'] = rank_df['SD_log'].rank(ascending=False, method='min')
rank_df['Rank_AitchVar'] = rank_df['AitchisonVar'].rank(ascending=False, method='min')

# --- Fill missing ranks with worst possible rank + 1 ---
max_rank = max(
    rank_df['Rank_CV'].max(skipna=True),
    rank_df['Rank_SDlog'].max(skipna=True),
    rank_df['Rank_AitchVar'].max(skipna=True)
)
rank_cols = ['Rank_CV', 'Rank_SDlog', 'Rank_AitchVar']
rank_df[rank_cols] = rank_df[rank_cols].fillna(max_rank + 1)

# --- Calculate mean rank and select top 20 elements ---
rank_df['MeanRank'] = rank_df[rank_cols].mean(axis=1)
top20_meanrank = rank_df.nsmallest(20, 'MeanRank').reset_index(drop=True)

print("🏆 Top 20 elements by mean rank (integrated variability score):")
display(top20_meanrank[['Element', 'CV_%', 'SD_log', 'AitchisonVar',
                        'Rank_CV', 'Rank_SDlog', 'Rank_AitchVar', 'MeanRank']])

### Block 9: Exporting Summary and Visualizing Top Elements

In [ ]:
# --- Export combined metrics and top-20 by mean rank ---
cmp_path = OUT_DIR_CSV / "metrics_combined_cv_sdlog_aitch.csv"
top_path = OUT_DIR_CSV / "selected_top20_by_meanrank.csv"

cv_comparison.to_csv(cmp_path, index=False, encoding="utf-8-sig")
top20_meanrank[['Element', 'CV_%', 'SD_log', 'AitchisonVar', 'MeanRank']].to_csv(
    top_path, index=False, encoding="utf-8-sig"
)

print("📁 Export completed:")
print(f" • Combined metrics → {cmp_path}")
print(f" • TOP-20 (MeanRank) → {top_path}")

# --- Normalized comparison plot of top-20 elements ---
common_idx = top20_meanrank['Element']
cmp20 = (
    cv_raw_df.join(sd_log_df, how='outer')
    .join(aitch_var_df, how='outer')
).loc[common_idx]

cmp20.columns = ['Robust CV (%)', 'SD(log x)', 'AitchisonVar']
norm20 = (cmp20 - cmp20.min()) / (cmp20.max() - cmp20.min())

ax = norm20.plot(kind='bar', figsize=(12, 6))
ax.set_title('Normalized variability metrics for top-20 elements')
ax.set_ylabel('Normalized value [0..1]')
ax.set_xlabel('Element')
ax.tick_params(axis='x', rotation=45)
ax.grid(axis='y', linestyle=':', alpha=0.5)
plt.tight_layout()

out_png = OUT_DIR_PNG / 'cmp20_metrics_normalized.png'
plt.savefig(out_png, dpi=300, bbox_inches='tight', facecolor='white', transparent=False)
plt.show()
plt.close()

print(f"✅ PNG saved: {out_png.resolve()}")

### Block 10: Correlation Analysis and Heatmap

In [ ]:
# --- Parameters for correlation analysis ---
CORR_METHOD = "spearman"
CORR_CUTOFF = 0.70  # Threshold for strong correlation

# --- Use top-20 elements from previous ranking as base ---
pre_selected_elements = top20_meanrank['Element'].tolist()

# --- Full correlation matrix (Spearman) ---
correlation_matrix = df_elems_ppm_named.corr(method=CORR_METHOD)

# --- Find additional elements strongly correlated with base elements ---
additional_elements = set()
for base_el in pre_selected_elements:
    if base_el in correlation_matrix.columns:
        col = correlation_matrix[base_el]
        mask = (col.abs() >= CORR_CUTOFF) & (col.index != base_el)
        correlated = col[mask]
        for el in correlated.index:
            if el not in pre_selected_elements:
                additional_elements.add(el)

print(f"➕ Additional elements with |ρ| ≥ {CORR_CUTOFF}:")
print(sorted(additional_elements) if additional_elements else "— none —")

# --- Prepare list of elements for heatmap ---
sel_exist = [c for c in pre_selected_elements if c in df_elems_ppm_named.columns]
add_exist = [c for c in additional_elements if c in df_elems_ppm_named.columns]
all_for_heatmap = sel_exist + [c for c in add_exist if c not in sel_exist]

# --- Subset correlation matrix and plot heatmap ---
corr_sub = df_elems_ppm_named[all_for_heatmap].corr(method=CORR_METHOD)

plt.figure(figsize=(0.6 * len(all_for_heatmap) + 4, 0.6 * len(all_for_heatmap) + 2))
sns.heatmap(corr_sub, annot=True, cmap='coolwarm', fmt=".2f",
            cbar_kws={'shrink': 0.8}, square=True)
plt.title("Spearman Correlation Matrix for Selected Elements")
plt.tight_layout()

heatmap_path = OUT_DIR_PNG / 'correlationMatrix_preSelEl.png'
plt.savefig(heatmap_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print(f"✅ Heatmap saved: {heatmap_path.resolve()}")

### Block 11: Anomaly Thresholds and Element Classification

In [ ]:
# --- Summary table: anomaly thresholds and counts ---
summary_rows = []
for el in all_for_heatmap:
    s = pd.to_numeric(df_elems_ppm_named[el], errors="coerce").dropna()
    s_pos = s[s > 0]
    if s_pos.empty:
        summary_rows.append({
            "Element": el,
            "Median (BG)": np.nan,
            "Geometric Mean (BG)": np.nan,
            "Anomaly Threshold (μ+2σ)": np.nan,
            "Anomaly Threshold (95th pct)": np.nan,
            "Anomaly Count (> μ+2σ)": 0
        })
        continue
    log_vals = np.log(s_pos)
    mu = log_vals.mean()
    sigma = log_vals.std(ddof=1)
    thr_log = float(np.exp(mu + 2.0 * sigma))
    thr_p95 = float(np.percentile(s_pos, 95))
    anomalies = s_pos[s_pos > thr_log]
    summary_rows.append({
        "Element": el,
        "Median (BG)": round(float(np.median(s_pos)), 4),
        "Geometric Mean (BG)": round(float(np.exp(mu)), 4),
        "Anomaly Threshold (μ+2σ)": round(thr_log, 4),
        "Anomaly Threshold (95th pct)": round(thr_p95, 4),
        "Anomaly Count (> μ+2σ)": int(len(anomalies))
    })

BG_AN_summary_df = pd.DataFrame(summary_rows)
display(BG_AN_summary_df.sort_values("Anomaly Count (> μ+2σ)", ascending=False))

# --- Derived metrics ---
BG_AN_summary_df["k_BG"] = BG_AN_summary_df["Anomaly Threshold (μ+2σ)"] / BG_AN_summary_df["Median (BG)"]
BG_AN_summary_df["tail_gap_R"] = BG_AN_summary_df["Anomaly Threshold (μ+2σ)"] / BG_AN_summary_df["Anomaly Threshold (95th pct)"]

# --- Classification rules ---
MIN_ANOM_COUNT = 15
MAX_TAIL_GAP = 1.30
TAIL_TOL = 0.05
HIGH_K_BG = 8.0
BASE_K_BG = 3.0

mask_strict = (
    (BG_AN_summary_df["Anomaly Count (> μ+2σ)"] >= MIN_ANOM_COUNT) &
    (BG_AN_summary_df["tail_gap_R"] <= MAX_TAIL_GAP) &
    (BG_AN_summary_df["k_BG"] >= BASE_K_BG)
)

mask_soft = (
    (BG_AN_summary_df["Anomaly Count (> μ+2σ)"] >= MIN_ANOM_COUNT) &
    (BG_AN_summary_df["tail_gap_R"] > MAX_TAIL_GAP) &
    (BG_AN_summary_df["tail_gap_R"] <= MAX_TAIL_GAP + TAIL_TOL) &
    (BG_AN_summary_df["k_BG"] >= HIGH_K_BG)
)

mask_core = mask_strict | mask_soft
mask_optional = ~mask_core & (BG_AN_summary_df["Anomaly Count (> μ+2σ)"] >= (MIN_ANOM_COUNT - 1))

BG_AN_summary_df["Group"] = np.where(
    mask_core, "core",
    np.where(mask_optional, "optional", "excluded")
)

# --- Print classification results ---
core_elements = BG_AN_summary_df.loc[BG_AN_summary_df["Group"] == "core", "Element"].tolist()
optional_elements = BG_AN_summary_df.loc[BG_AN_summary_df["Group"] == "optional", "Element"].tolist()
excluded_elements = BG_AN_summary_df.loc[BG_AN_summary_df["Group"] == "excluded", "Element"].tolist()

print("✅ Core anomaly elements:", ", ".join(core_elements) if core_elements else "— none —")
print("ℹ️ Optional elements:", ", ".join(optional_elements) if optional_elements else "— none —")
print("❌ Excluded elements:", ", ".join(excluded_elements) if excluded_elements else "— none —")

### Block 12: Element Scoring — Integrated Indicative Index

In [ ]:
# --- Normalize metrics for scoring ---
TAIL_CAP = 0.30  # cap for tail deviation penalty
W_ANOM, W_CONT, W_TAIL = 0.50, 0.30, 0.20  # weights for scoring

anom_max = BG_AN_summary_df["Anomaly Count (> μ+2σ)"].replace(0, np.nan).max()
cont_max = BG_AN_summary_df["k_BG"].replace(0, np.nan).max()

BG_AN_summary_df["I_anom"] = BG_AN_summary_df["Anomaly Count (> μ+2σ)"] / anom_max
BG_AN_summary_df["I_cont"] = BG_AN_summary_df["k_BG"] / cont_max

# Tail stability: closer to R=1 is better
abs_dev = (BG_AN_summary_df["tail_gap_R"] - 1.0).abs()
BG_AN_summary_df["I_tail"] = 1.0 - np.minimum(abs_dev, TAIL_CAP) / TAIL_CAP

# Final indicative score
BG_AN_summary_df["S_elem"] = (
    W_ANOM * BG_AN_summary_df["I_anom"] +
    W_CONT * BG_AN_summary_df["I_cont"] +
    W_TAIL * BG_AN_summary_df["I_tail"]
)

# Strip '_ppm' suffix for cleaner display
def strip_suffix(s: str, suffix="_ppm") -> str:
    return s[:-len(suffix)] if isinstance(s, str) and s.endswith(suffix) else s

BG_AN_summary_df["Element_symbol"] = BG_AN_summary_df["Element"].apply(strip_suffix)

# Sort and export
BG_AN_summary_df_sorted = BG_AN_summary_df.sort_values("S_elem", ascending=False)
summary_path = OUT_DIR_CSV / "elements_summary_with_scores.csv"
BG_AN_summary_df_sorted.to_csv(summary_path, index=False, encoding="utf-8-sig")

print(f"📁 Element scoring summary exported: {summary_path.resolve()}")
display(BG_AN_summary_df_sorted[["Element", "S_elem", "Group"]].head(20))

### Block 13: Aggregation by Mineralization Types

In [ ]:
# --- Define mineralization types and associated elements ---
types = {
    "Zone 1 (Cu–Mo–W–Bi)": ["Cu", "Mo", "W", "Bi"],
    "Zone 2 (Pb–Zn–Sb–Mn)": ["Pb", "Zn", "Sb", "Mn"],
    "Zone 3 (Au–Ag–Sb–Se)": ["Au", "Ag", "Sb", "Se"]
} # specify your own complex of elements

agg_rows = []
for tname, els in types.items():
    sub = BG_AN_summary_df[BG_AN_summary_df["Element_symbol"].isin(els)]
    if sub.empty:
        agg_rows.append({
            "Type": tname,
            "Elements Included": 0,
            "Σ Anomalies": 0,
            "Mean Anomaly Intensity": np.nan,
            "Mean Contrast": np.nan,
            "Mean Tail Stability": np.nan,
            "Mean Indicative Score": np.nan,
            "Total Indicative Score": 0.0
        })
        continue
    agg_rows.append({
        "Type": tname,
        "Elements Included": len(sub),
        "Σ Anomalies": int(sub["Anomaly Count (> μ+2σ)"].sum()),
        "Mean Anomaly Intensity": float(sub["I_anom"].mean()),
        "Mean Contrast": float(sub["I_cont"].mean()),
        "Mean Tail Stability": float(sub["I_tail"].mean()),
        "Mean Indicative Score": float(sub["S_elem"].mean()),
        "Total Indicative Score": float(sub["S_elem"].sum())
    })

df_types = pd.DataFrame(agg_rows)
df_types_sum = df_types.sort_values("Total Indicative Score", ascending=False)
df_types_avg = df_types.sort_values("Mean Indicative Score", ascending=False)

# --- Export aggregated tables ---
df_types_sum.to_csv(OUT_DIR_CSV / "types_scores_sorted_by_sum.csv", index=False, encoding="utf-8-sig")
df_types_avg.to_csv(OUT_DIR_CSV / "types_scores_sorted_by_mean.csv", index=False, encoding="utf-8-sig")

print("\n🏁 Mineralization types (sorted by TOTAL score):")
print(df_types_sum.to_string(index=False))

print("\n🏁 Mineralization types (sorted by MEAN score):")
print(df_types_avg.to_string(index=False))


# --- Barplot: total indicative score by mineralization type ---
plt.figure(figsize=(9, 4.5))
sns.barplot(data=df_types_sum, x="Total Indicative Score", y="Type", color="#4C9F70")
plt.title("Total Contribution by Mineralization Type (Sum of Scores)")
plt.xlabel("Total Indicative Score")
plt.ylabel("")
plt.tight_layout()

sum_plot_path = OUT_DIR_PNG / "types_sum_score.png"
plt.savefig(sum_plot_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

# --- Barplot: mean indicative score by mineralization type ---
plt.figure(figsize=(9, 4.5))
sns.barplot(data=df_types_avg, x="Mean Indicative Score", y="Type", color="#3C7DD9")
plt.title("Signal Intensity by Mineralization Type (Mean Score)")
plt.xlabel("Mean Indicative Score")
plt.ylabel("")
plt.tight_layout()

mean_plot_path = OUT_DIR_PNG / "types_mean_score.png"
plt.savefig(mean_plot_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print(f"✅ Saved plots:")
print(f" • Total score → {sum_plot_path.resolve()}")
print(f" • Mean score → {mean_plot_path.resolve()}")

### Block 15: Final Element Selection and Level Rounding Configuration

In [ ]:
# --- Final selection of informative elements for mapping ---
selected_elements = [
    'Cu_ppm', 'Au_ppb', 'Ag_ppm', 'Mo_ppm', 'W_ppm', 'Bi_ppm',
    'Sb_ppm', 'Sn_ppm', 'Pb_ppm', 'Zn_ppm', 'Mn_ppm', 'Se_ppm'
]

# --- Rounding steps for map readability ---
ELEMENTS_ROUND_STEP = {
    'Cu_ppm': 50.0,
    'Au_ppb': 10.0,
    'Ag_ppm': 0.5,
    'Mo_ppm': 1.0,
    'W_ppm': 5.0,
    'Bi_ppm': 1.0,
    'Sb_ppm': 1.0,
    'Sn_ppm': 1.0,
    'Pb_ppm': 100.0,
    'Zn_ppm': 100.0,
    'Mn_ppm': 500.0,
    'Se_ppm': 2.0
}

print("✅ Selected informative elements and rounding steps configured.")

### Block 16: Mapping Configuration and Preparation

In [ ]:
# --- Configuration for mapping pipeline ---
from dataclasses import dataclass
from typing import Optional
from decimal import Decimal

# Smoothing and interpolation
from scipy.ndimage import gaussian_filter
from scipy.interpolate import griddata

# Geometry and GIS
import geopandas as gpd
from shapely.geometry import (
    Polygon, MultiPolygon, Point, LineString, MultiPoint, LinearRing
)
from shapely.ops import unary_union
from shapely.prepared import prep as prep_geom
from shapely.strtree import STRtree
from shapely.geometry.base import JOIN_STYLE

# Contour extraction from masks
from skimage import measure

@dataclass
class Config:
    # Coordinate system for export
    crs_epsg: str = "EPSG:00000"  # WGS 84 / INSERT HERE YOUR OWN COORDINATE SYSTEM

    # Sigma level configuration
    sigma_step: float = 0.25
    winsorize_enabled: bool = True
    winsor_k_sigma: float = 3.0

    # Edge anomaly rounding and view padding
    buffer_m: float = 200.0
    view_padding_m: float = 400.0
    ring_spacing_m: float = 50.0

    # Interpolation and smoothing
    grid_n: int = 500
    smooth_m: float = 60.0

    # Polygon smoothing (in/out buffer)
    smooth_radius_m: float = 60.0

    # Chaikin smoothing parameters
    b_maxseg_m: float = 15.0
    b_iter: int = 2

    # Minimum polygon area (m²)
    min_area_m2: float = 7000.0

    # SHP attribute encoding
    encoding: str = "UTF-8"

CFG = Config()
print("✅ Mapping pipeline configuration initialized.")

###  Block 17: Mapping Functions — Interpolation, Smoothing, Contour Extraction, Export

#### Block 17.1: Utility Functions for Mapping

In [ ]:
# --- Determine decimal places from rounding step ---
def decimals_from_step(step: float) -> int:
    """
    Determines number of decimal places needed for a given rounding step.
    """
    d = Decimal(str(step)).as_tuple().exponent
    return max(0, -d)

#### Block 17.2: Winsorization in Log-space

In [ ]:
def winsorize_log(z: np.ndarray, k: float) -> np.ndarray:
    """
    Soft clipping of tails: limits log(z) to [μ−kσ, μ+kσ], returns values in linear scale.
    """
    zp = np.asarray(z, float)
    msk = zp > 0
    if msk.sum() < 2:
        return zp
    logz = np.log(zp[msk])
    mu, sd = float(np.mean(logz)), float(np.std(logz, ddof=1))
    lo, hi = mu - k * sd, mu + k * sd
    logz = np.clip(logz, lo, hi)
    zp[msk] = np.exp(logz)
    return zp

#### Block 17.3: Merge Nearby Sample Points

In [ ]:
# --- Merge nearby sample points ---
def merge_close_points(x: np.ndarray, y: np.ndarray, z: np.ndarray, radius_m: float):
    """
    Merges spatially close samples within 'radius_m' using a union-find over pairwise neighbors.
    Returns averaged x, y and median z per cluster.

    Parameters
    ----------
    x, y : arrays of coordinates (meters, projected CRS)
    z : array of values (positive)
    radius_m : clustering radius in meters

    Returns
    -------
    xv2, yv2, zv2 : numpy arrays of merged coordinates and values
    """
    from scipy.spatial import cKDTree

    coords = np.c_[x, y]
    if len(coords) == 0:
        return x, y, z

    tree = cKDTree(coords)
    pairs = tree.query_pairs(r=radius_m)

    parent = list(range(len(coords)))

    def find(a):
        while parent[a] != a:
            parent[a] = parent[parent[a]]
            a = parent[a]
        return a

    def union(a, b):
        ra, rb = find(a), find(b)
        if ra != rb:
            parent[rb] = ra

    for i, j in pairs:
        union(i, j)

    clusters = {}
    for i in range(len(coords)):
        r = find(i)
        clusters.setdefault(r, []).append(i)

    xv2, yv2, zv2 = [], [], []
    for ids in clusters.values():
        xv2.append(float(np.mean(x[ids])))
        yv2.append(float(np.mean(y[ids])))
        zv2.append(float(np.median(z[ids])))

    return np.array(xv2), np.array(yv2), np.array(zv2)

#### Block 17.4: Sigma Level Generation and Labeling

In [ ]:
# --- Sigma label for a contour level ---
def sigma_label_for_level(L: float, mu: float, sigma: float, step: float) -> str:
    k_val = (np.log(L) - mu) / sigma
    k_val = float(np.round(k_val / step) * step)
    if abs(k_val - int(k_val)) < 1e-9:
        return f"{int(k_val)}σ"
    k_str = f"{k_val:.2f}".rstrip("0").rstrip(".")
    return f"{k_str}σ"


# --- Generate sigma-based contour levels in linear space ---
def make_sigma_levels(mu: float, sigma: float, zmax: float,
                      sigma_step: float, round_step: float) -> list[float]:
    k = sigma_step
    raw = []
    while True:
        L = float(np.exp(mu + k * sigma))
        if L > zmax: break
        raw.append(L); k += sigma_step
    if not raw: return []
    levels = np.unique(np.round(np.array(raw) / round_step) * round_step)
    return [float(v) for v in levels if 0 < v <= zmax]

#### Block 17.5: Extract polygons from a boolean mask (with holes)

In [ ]:
def polygons_from_mask(mask_bool: np.ndarray, GX: np.ndarray, GY: np.ndarray):
    """
    Converts a boolean exceedance mask (ny x nx) into Shapely polygons using image contours.
    Ensures correct assignment of interior holes to outer rings.

    Parameters
    ----------
    mask_bool : 2D boolean array
    GX, GY    : 2D arrays of the same shape with grid x/y in map units

    Returns
    -------
    list of shapely.Polygon (holes preserved) and/or MultiPolygon pieces
    """
    assert mask_bool.shape == GX.shape == GY.shape, "Mask and grid shapes must match"
    ny, nx = mask_bool.shape
    x = GX[0, :]
    y = GY[:, 0]

    labels, nlabels = measure.label(mask_bool.astype(np.uint8), connectivity=2, return_num=True)
    polys_out = []

    def _img_to_world(contour_rc):
        rows, cols = contour_rc[:, 0], contour_rc[:, 1]
        xs = np.interp(cols, np.arange(nx), x)
        ys = np.interp(rows, np.arange(ny), y)
        return np.c_[xs, ys]

    def _signed_area_img(contour_rc):
        pts = np.c_[contour_rc[:, 1], contour_rc[:, 0]]
        return 0.5 * np.sum(pts[:-1, 0] * pts[1:, 1] - pts[1:, 0] * pts[:-1, 1])

    for lbl in range(1, nlabels + 1):
        roi = (labels == lbl)
        if roi.sum() < 4:
            continue

        contours = measure.find_contours(roi.astype(np.uint8), 0.5)
        if not contours:
            continue

        outers_img, holes_img = [], []
        for c in contours:
            if len(c) < 3:
                continue
            A = _signed_area_img(c)
            (outers_img if A > 0 else holes_img).append(c)

        if not outers_img:
            # Fallback: treat the largest area as the outer
            areas = [abs(_signed_area_img(c)) for c in contours]
            outer_idx = int(np.argmax(areas))
            outers_img = [contours[outer_idx]]
            holes_img = [c for i, c in enumerate(contours) if i != outer_idx]

        outers_world = [_img_to_world(c) for c in outers_img]
        holes_world  = [_img_to_world(c) for c in holes_img]

        outer_geoms = [Polygon(outer) for outer in outers_world]
        hole_geoms  = [Polygon(h) for h in holes_world]

        if outer_geoms:
            outer_tree = STRtree(outer_geoms)
            holes_by_outer = {id(g): [] for g in outer_geoms}

            # Assign each hole to the containing outer polygon
            for h in hole_geoms:
                cand = outer_tree.query(h.centroid)
                # 'cand' can be indices or geometries depending on env; normalize to list of geoms
                if isinstance(cand, (list, tuple)):
                    candidates = cand
                else:
                    try:
                        candidates = [outer_geoms[int(i)] for i in np.atleast_1d(cand)]
                    except Exception:
                        candidates = outer_geoms

                for o in candidates:
                    if o.contains(h):
                        holes_by_outer[id(o)].append(h)
                        break
            for o in outer_geoms:
                holes_coords = [
                    h.exterior.coords[:] for h in holes_by_outer[id(o)]
                    if h.exterior and len(h.exterior.coords) >= 4
                ]
                poly = Polygon(o.exterior.coords[:], holes=holes_coords).buffer(0)
                if not poly.is_empty and poly.is_valid:
                    polys_out.append(poly)

    return polys_out

#### Block 17.6: Geometric smoothing (Chaikin corner cutting)

In [ ]:
# --- Densify a LinearRing to limit max segment length (meters) ---
def _densify_ring(line: LinearRing, maxseg_m: float) -> LineString:
    L = float(line.length)
    n = max(8, int(np.ceil(L / maxseg_m)))
    pts = [line.interpolate(i / n, normalized=True) for i in range(n)]
    pts.append(pts[0])
    return LineString(pts)

# --- One iteration of Chaikin smoothing (closed ring supported) ---
def _chaikin(coords: np.ndarray, keep_closed: bool = True) -> np.ndarray:
    P = np.asarray(coords, float)
    closed = keep_closed and np.allclose(P[0], P[-1])
    if closed:
        P = P[:-1]
    Q = []
    for i in range(len(P)):
        p0 = P[i]
        p1 = P[(i + 1) % len(P)]
        Q.append(0.75 * p0 + 0.25 * p1)
        Q.append(0.25 * p0 + 0.75 * p1)
    Q = np.vstack(Q)
    if keep_closed:
        Q = np.vstack([Q, Q[0]])
    return Q

# --- Chaikin smoothing for a polygon (exterior + interiors) ---
def smooth_polygon_chaikin(poly: Polygon, maxseg_m: float = 15.0, n_iter: int = 2) -> Polygon:
    # Exterior
    ext = _densify_ring(LinearRing(poly.exterior.coords), maxseg_m)
    ext_coords = np.array(ext.coords)
    for _ in range(max(1, int(n_iter))):
        ext_coords = _chaikin(ext_coords, keep_closed=True)

    # Interiors (holes)
    holes_coords = []
    for hole in poly.interiors:
        hh = _densify_ring(LinearRing(hole.coords), maxseg_m)
        hc = np.array(hh.coords)
        for _ in range(max(1, int(n_iter - 1))):
            hc = _chaikin(hc, keep_closed=True)
        if len(hc) >= 4:
            holes_coords.append(hc)

    return Polygon(ext_coords, holes=holes_coords).buffer(0)

#### Block 17.7: Element map plotting (filled contours + lines, points, buffer, scalebar)

In [ ]:
# --- Plot element map (filled contours, isolines, samples, buffer, scalebar) ---
def plot_element_map(
    GX, GY, GZ_final, levels, hull_buf, xv, yv,
    title: str, element_name: str,
    out_png: Optional[Path] = None,
    buffer_m: float = 0.0, smooth_m: float = 0.0,
    cmap: str = "turbo", value_decimals: int = 0
):
    """
    Draws filled contours + isolines for 'GZ_final' clipped by hull buffer; overlays sample points,
    buffer outline, simple north arrow and a metric scalebar.
    """
    # Safe colormap
    try:
        plt.get_cmap(cmap)
    except Exception:
        cmap = "viridis"

    fig, ax = plt.subplots(figsize=(12, 9.5))

    mgrid = np.ma.masked_invalid(GZ_final)
    cf = ax.contourf(GX, GY, mgrid, levels=levels, cmap=cmap, alpha=0.72, antialiased=True)
    cs = ax.contour(GX, GY, mgrid, levels=levels, colors="k", linewidths=0.45)

    # Optional labels on a subset of isolines
    if len(levels) >= 1:
        try:
            step = max(1, len(levels) // 8)  # avoid clutter
            ax.clabel(cs, levels=levels[::step], fmt=lambda v: f"{int(v)}", inline=True, fontsize=8)
        except Exception:
            pass

    # Samples and buffer outline
    ax.scatter(xv, yv, c="black", s=6, edgecolors="none", alpha=0.55, zorder=3, label="Samples")
    rx, ry = hull_buf.exterior.xy
    ax.plot(rx, ry, color="white", lw=1.4, ls="--", alpha=0.9, label=f"Convex hull +{int(buffer_m)} m")

    ax.set_aspect("equal")
    ax.set_xlabel("East (m)"); ax.set_ylabel("North (m)")
    ax.set_title(title)
    ax.legend(loc="lower right", frameon=True)
    ax.set_xticklabels([]); ax.set_yticklabels([])

    # Simple north arrow
    ax.annotate('N', xy=(0.95, 0.95), xytext=(0.95, 0.85),
                xycoords='axes fraction', textcoords='axes fraction',
                arrowprops=dict(facecolor='black', width=2, headwidth=10),
                ha='center', va='center', fontsize=12, fontweight='bold')

    # Colorbar
    cbar = fig.colorbar(cf, ax=ax)
    cbar.set_label(f"{element_name}: rounded σ-levels")

    # Metric scalebar (3 segments of equal length)
    scale_origin_x = xv.min() + (xv.max() - xv.min()) / 2 - 300
    scale_origin_y = yv.min() - 50
    segment_length = 400  # meters per segment
    segment_height = 50

    for i in range(3):
        color = 'black' if i % 2 == 0 else 'white'
        ax.add_patch(
            Rectangle(
                (scale_origin_x + i * segment_length, scale_origin_y),
                segment_length, segment_height,
                facecolor=color, edgecolor='black', zorder=4
            )
        )
    total_length = segment_length * 3
    ax.add_patch(
        Rectangle((scale_origin_x, scale_origin_y), total_length, segment_height,
                  fill=False, edgecolor='black', linewidth=1.2, zorder=5)
    )
    for i in range(4):
        ax.text(scale_origin_x + i * segment_length, scale_origin_y - 15,
                f"{i * segment_length}", ha='center', va='top', fontsize=10)
    ax.text(scale_origin_x + total_length + 20, scale_origin_y + segment_height / 2,
            "m", ha='left', va='center', fontsize=9)

    plt.tight_layout()
    if out_png is not None:
        fig.savefig(out_png, dpi=300, bbox_inches="tight", facecolor="white")
    plt.show()
    plt.close(fig)

#### Block 17.8: Export polygons (to SHP) and the PNG map

In [ ]:
# --- Export SHP/PNG for a set of polygons at multiple levels ---
def export_records(
    records: list[dict],
    GX, GY, GZ_final, levels, hull_buf, xv, yv,
    element_name: str,
    cfg: Config,
    out_png_path: Optional[Path] = None,
    title_suffix: str = "",
    value_decimals: int = 0
):
    """
    Builds a GeoDataFrame from 'records' and writes:
      - ESRI Shapefile with attributes (Element, Unit, Value, SigmaK, Area_m2/km2)
      - PNG map via 'plot_element_map' if 'out_png_path' is provided.
    """
    if not records:
        print(f"⚠️ No polygons created for {element_name}. Skipping export.")
        return

    crs_str = str(cfg.crs_epsg) if cfg.crs_epsg else None
    gdf = gpd.GeoDataFrame(records, geometry="geometry", crs=crs_str)

    # Normalize attribute dtypes
    gdf["Element"] = gdf["Element"].astype(str)
    gdf["Unit"] = gdf["Unit"].astype(str)
    gdf["Value"] = gdf["Value"].astype("float64")
    gdf["SigmaK"] = gdf["SigmaK"].astype(str)

    # Explode multipolygons
    gdf = gdf.explode(index_parts=False, ignore_index=True)

    # Paths
    shp_path = OUT_DIR_SHP / f"{element_name}_contours_B_chaikin.shp"
    png_path = out_png_path or (OUT_DIR_PNG / f"{element_name}_sigma_rounded_B_chaikin.png")

    # Export SHP
    gdf.to_file(shp_path, driver="ESRI Shapefile", encoding=cfg.encoding)

    # Compose title and export PNG
    title = (
        f"{element_name} : rounded anomaly levels\n"
        f"Interpolation +{int(cfg.buffer_m)} m; BG = geom. mean; smoothing ≈{int(cfg.smooth_m)} m"
    )
    plot_element_map(
        GX, GY, GZ_final, levels, hull_buf, xv, yv,
        title=title, element_name=element_name,
        out_png=png_path,
        buffer_m=cfg.buffer_m, smooth_m=cfg.smooth_m, cmap="turbo",
        value_decimals=value_decimals
    )
    print(f"✅ SHP saved:  {shp_path.resolve()}")
    print(f"✅ PNG saved:  {png_path.resolve()}")

#### Block 17.9: End‑to‑end processing for one element

In [ ]:
# --- Full mapping pipeline for a single element ---
def process_element(
    df: pd.DataFrame,
    element_name: str,
    round_step_value: float,
    cfg: Config = CFG,
    merge_nearby_enabled: bool = True,
    merge_radius_m: float = 25.0
):
    """
    End-to-end workflow for one element:
      1) Filter positive values, optionally merge close samples and winsorize tails (log-space).
      2) Estimate mu, sigma in log-space; generate sigma levels and round to 'round_step_value'.
      3) Interpolate to regular grid with soft boundary (background = geometric mean).
      4) Gaussian smoothing (approx. 'smooth_m' meters).
      5) Extract polygons per level, apply buffer-in/out rounding and Chaikin smoothing.
      6) Export SHP and a single PNG map.

    Notes
    -----
    - Requires 'x', 'y' coordinate columns in meters (projected CRS).
    - 'round_step_value' should match human-readable step for the element (e.g., 50 ppm).
    """
    if element_name not in df.columns:
        print(f"⚠️ Column '{element_name}' not found. Skipping.")
        return

    # Coordinates and values
    x = pd.to_numeric(df["x"], errors="coerce").to_numpy()
    y = pd.to_numeric(df["y"], errors="coerce").to_numpy()
    z = pd.to_numeric(df[element_name], errors="coerce").to_numpy()
    mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(z) & (z > 0)
    xv, yv, zv = x[mask], y[mask], z[mask]

    if len(zv) < 10:
        print(f"⚠️ Not enough points for '{element_name}' ({len(zv)} < 10). Skipping.")
        return

    # Optional pre-processing
    if merge_nearby_enabled and merge_radius_m > 0:
        xv, yv, zv = merge_close_points(xv, yv, zv, merge_radius_m)
    if cfg.winsorize_enabled:
        zv = winsorize_log(zv, cfg.winsor_k_sigma)

    # Log-space stats
    log_vals = np.log(zv)
    mu = float(np.mean(log_vals))
    sigma = float(np.std(log_vals, ddof=1))
    z_max = float(np.max(zv))
    geo_mean = float(np.exp(mu))

    # Sigma levels (rounded)
    levels = make_sigma_levels(mu, sigma, z_max, cfg.sigma_step, round_step_value)
    if not levels:
        print(f"⚠️ No σ-levels for '{element_name}'. Skipping.")
        return
    decimals = decimals_from_step(round_step_value)

    # Convex hull buffer and background ring
    hull = MultiPoint(list(zip(xv, yv))).convex_hull
    hull_buf = hull.buffer(cfg.buffer_m, resolution=64, join_style=JOIN_STYLE.round)
    ring = LineString(hull_buf.exterior.coords)
    perim = ring.length
    n_pts = max(8, int(math.ceil(perim / cfg.ring_spacing_m)))
    ring_x, ring_y = [], []
    for i in range(n_pts):
        p = ring.interpolate((i / n_pts) * perim)
        ring_x.append(p.x); ring_y.append(p.y)
    ring_val = geo_mean
    ring_z = np.full(len(ring_x), ring_val, dtype=float)

    # Merge real and background samples
    x_ext = np.concatenate([xv, np.array(ring_x)])
    y_ext = np.concatenate([yv, np.array(ring_y)])
    z_ext = np.concatenate([zv, ring_z])

    # Grid and interpolation
    xmin, xmax = xv.min() - cfg.view_padding_m, xv.max() + cfg.view_padding_m
    ymin, ymax = yv.min() - cfg.view_padding_m, yv.max() + cfg.view_padding_m
    gx = np.linspace(xmin, xmax, cfg.grid_n)
    gy = np.linspace(ymin, ymax, cfg.grid_n)
    GX, GY = np.meshgrid(gx, gy)

    try:
        GZ = griddata((x_ext, y_ext), z_ext, (GX, GY), method="cubic")
    except Exception:
        GZ = griddata((x_ext, y_ext), z_ext, (GX, GY), method="linear")
    GZ_near = griddata((x_ext, y_ext), z_ext, (GX, GY), method="nearest")
    GZ = np.where(np.isfinite(GZ), GZ, GZ_near)

    # Inside-buffer mask (soft boundary)
    try:
        prep_buf = prep_geom(hull_buf)
    except Exception:
        prep_buf = None
    inside_mask = np.zeros(GX.shape, dtype=bool)
    for i in range(GX.shape[0]):
        row_pts = [Point(float(GX[i, j]), float(GY[i, j])) for j in range(GX.shape[1])]
        if prep_buf is not None:
            try:
                inside_mask[i, :] = [prep_buf.covers(pt) for pt in row_pts]
            except Exception:
                inside_mask[i, :] = [hull_buf.covers(pt) for pt in row_pts]
        else:
            inside_mask[i, :] = [hull_buf.covers(pt) for pt in row_pts]

    # Gaussian smoothing (convert meters to grid std-dev)
    dx = (xmax - xmin) / (cfg.grid_n - 1)
    dy = (ymax - ymin) / (cfg.grid_n - 1)
    sigx = max(0.1, cfg.smooth_m / dx)
    sigy = max(0.1, cfg.smooth_m / dy)

    WZ = GZ.copy()
    WZ[~inside_mask] = ring_val  # soft background
    WZ_smooth = gaussian_filter(WZ, sigma=(sigy, sigx), mode="nearest")
    GZ_final = np.where(inside_mask, WZ_smooth, np.nan)

    # Polygonize per level, geometric rounding + Chaikin smoothing
    unit = parse_unit(element_name) if "parse_unit" in globals() else ""
    records = []
    for L in levels:
        base_mask = np.isfinite(GZ_final) & (GZ_final >= L)
        polys_L = polygons_from_mask(base_mask, GX, GY)
        if not polys_L:
            continue
        sigma_lbl = sigma_label_for_level(L, mu, sigma, step=cfg.sigma_step)

        for poly in polys_L:
            # Geometric rounding of corners via in/out buffer
            if cfg.smooth_radius_m and cfg.smooth_radius_m > 0:
                poly = (
                    poly.buffer(cfg.smooth_radius_m, resolution=64, join_style=JOIN_STYLE.round)
                        .buffer(-cfg.smooth_radius_m, resolution=64, join_style=JOIN_STYLE.round)
                )

            # Chaikin smoothing + validity fix
            poly = smooth_polygon_chaikin(poly, maxseg_m=cfg.b_maxseg_m, n_iter=cfg.b_iter).buffer(0)

            # Keep only valid polygons above the area threshold
            geoms = [poly] if poly.geom_type == "Polygon" else list(poly.geoms)
            for p in geoms:
                p = p.buffer(0)
                if p.is_empty or not p.is_valid:
                    continue
                if cfg.min_area_m2 and float(p.area) < float(cfg.min_area_m2):
                    continue

                val_rounded = round(float(L), decimals)
                records.append({
                    "Element": str(element_name),
                    "Unit": unit,
                    "Value": float(val_rounded),
                    "SigmaK": str(sigma_lbl),
                    "Area_m2": float(p.area),
                    "Area_km2": float(p.area) / 1e6,
                    "geometry": p,
                })

    # Export SHP + PNG (single map per element)
    export_records(
        records,
        GX, GY, GZ_final, levels, hull_buf, xv, yv,
        element_name=element_name,
        cfg=cfg,
        out_png_path=None,
        title_suffix="",
        value_decimals=decimals,
    )

#### Block 17.A: Batch run over selected elements

In [ ]:
# --- Batch driver ---
assert 'x' in df.columns and 'y' in df.columns, "DataFrame must contain 'x' and 'y' columns."

for el in selected_elements:
    step_val = ELEMENTS_ROUND_STEP.get(el, None)
    if step_val is None:
        print(f"⚠️ No rounding step configured for '{el}'. Skipping.")
        continue
    try:
        print(f"\n=== Processing: {el} (round step {step_val}) ===")
        process_element(df, el, float(step_val), cfg=CFG, merge_nearby_enabled=True, merge_radius_m=25.0)
    except Exception as e:
        print(f"❌ Error while processing '{el}': {e}")

# Analysis and Export Completed ✅